<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/federated_learning_advanced_remote_tensoroperations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT - XI
## TO IMPLEMENT FEDERATED LEARNING ON TOY DATABASE 

### First we are going to train our model simply using toy dataset and then we will apply federated learning , thus using virtual workers to train our model on remote machines and simulate the training.

In [0]:
#Let's first install pysyft and import our packages
pip install syft

In [3]:
import torch as th
import numpy as np
import syft as sy
from torch import nn, optim

W0708 15:44:19.628088 140421954885504 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0708 15:44:19.650886 140421954885504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)


So, firstly , we will train the simple linear model on our local machine but his time "using optimizers"

Let's create simple toy dataset including the data as well their labels.

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

Do you see we have written inside tensor like this - [1., 1] , [0, 1] and so the 'dot' represented in the first one will convert the whole tensor into float tensor implicitly.

In [6]:
print(data.dtype)
print(target.dtype)

torch.float32
torch.float32


In [0]:
# define the model as it contains only two inputs and one output so we need only one linear layer 
model = nn.Linear(2, 1)

In [0]:
# let's use here SGD optimizer
optimizer = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
# let's define a function for training loop
def train(epochs):
  avg_loss = []
  for e in range(epochs): # iterating over the entire data 
    optimizer.zero_grad()   # clearing the gradients
    pred = model(data)   # calcualting output of model 
    loss = ((pred - target)**2).sum()  # calculating loss (mistakes)
    loss.backward()  # calculating gradients of error / loss function 
    optimizer.step()  # taking step in negative of gradient , gradient descent step
    print(loss.data)  # printing training loss / mistakes 
    avg_loss.append(loss.data)
  print(f"avg loss : {sum(avg_loss)/len(avg_loss)}")

In [10]:
train(20)

tensor(2.0617)
tensor(1.1503)
tensor(0.7433)
tensor(0.4915)
tensor(0.3273)
tensor(0.2193)
tensor(0.1479)
tensor(0.1004)
tensor(0.0687)
tensor(0.0473)
tensor(0.0329)
tensor(0.0230)
tensor(0.0162)
tensor(0.0115)
tensor(0.0083)
tensor(0.0060)
tensor(0.0043)
tensor(0.0032)
tensor(0.0023)
tensor(0.0017)
avg loss : 0.27335864305496216


Now, it's time for federated learning of this model 

In [0]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# let's start by creating and sending the datasets to the virtual workers we are going to create.
bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")

data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)
datasets = [(data_bob, target_bob), (data_alice, target_alice)]


In [0]:
def train_fed(epochs):
  model = nn.Linear(2,1)
  optimizer = optim.SGD(params=model.parameters(), lr=0.1)
  for e in range(epochs):
    for inputs, labels in datasets:
      model = model.send(inputs.location) # sending model to the location where tensors are located 
      optimizer.zero_grad()
      pred = model(inputs)    
      loss = ((pred - labels)**2).sum()  
      loss.backward()   
      optimizer.step()  
      model =  model.get() # taking the model back with updates
      
      print(loss.get())   

In [69]:
train_fed(20)

tensor(3.7706, requires_grad=True)
tensor(0.0499, requires_grad=True)
tensor(0.0577, requires_grad=True)
tensor(0.0421, requires_grad=True)
tensor(0.0280, requires_grad=True)
tensor(0.0325, requires_grad=True)
tensor(0.0204, requires_grad=True)
tensor(0.0245, requires_grad=True)
tensor(0.0152, requires_grad=True)
tensor(0.0185, requires_grad=True)
tensor(0.0114, requires_grad=True)
tensor(0.0139, requires_grad=True)
tensor(0.0085, requires_grad=True)
tensor(0.0105, requires_grad=True)
tensor(0.0063, requires_grad=True)
tensor(0.0079, requires_grad=True)
tensor(0.0047, requires_grad=True)
tensor(0.0059, requires_grad=True)
tensor(0.0035, requires_grad=True)
tensor(0.0045, requires_grad=True)
tensor(0.0026, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0008, requi

THIS FUNCTION HAS QUITE ISSUES IN CONTEXT TO DIFFERENTIAL PRIVACY MEASURES.
Now, let's understand this , let's say we are in every iteration , we are sending model to one worker at a time and then getting back the model and then doing this for next worker and so on.
Then this can be actually reversed engineer , so that we can find out who made the changes to the inputs we are feeding like in word embeddings etc...

Therefore , we have to find ways to mitigate this problem and apply measures of privacy to prevent the above issue , one way to do this is to train the model with different workers at same time and then finally getting model back with averaged updates to the server.
This will be quite difficult to find out who changed and influenced which weights because we have send the model to every worker at same time and then we are getting the model at the end of training with averaged updates from each of the workers.